### Carregando funções Turing

In [6]:
%run -i '/home/.Turing/TuringCredentialsAccess.py3'
%run -i '/home/.Turing/TuringLabFunctions.py3'

------------- Turing Lab Data Science --------------------
------ Funções carregadas em memória com sucesso ---------
--------- Data da última atualização: --------------------
--------------- 17/08/2019 -------------------------------
----------------------------------------------------------
--- Consulte o manual de Biblioteca de Funções Turing: ---


----------------------------------------------------------


#### Mapear bibliotecas Python necessárias

In [7]:
import pandas as pd
import numpy as np

### Parêmetros do Projeto

In [8]:
#---------- Definição dos Parâmetros do projeto ---------------------------------
nm_bucket = 'turing-bkt-treinamentos'
id_empresa = '1022'

# Id do projeto parametrizado no Turing (o mesmo valor dado ao campo Id Projeto do Timeline)
id_projeto = '10001'

# Nome da tabela utilizada para desenvolver o modelo
nm_file_desenv_orig = 'ABT01_TITANIC_DESENV.csv'

# Nome da tabela utilizada para validar o modelo (out-of-time ou out-of-sample)
nm_file_valida_orig = 'OOS01_TITANIC_VALID.csv'

# Nome da variável resposta (quando houve)
targetname = 'Survived'

# Delimitador da tabela 
abt_delimiter = ','

 

### Credenciais de acessos ao projeto

In [9]:
################### Não alterar este parágrafo #####################################
#---------- Gerando Credenciais para Acesso Seguro ---------------------------------
id_modelo = id_projeto
schema = 'EMP_'+ id_empresa +'_Proj_'+ id_projeto
S3fs,S3session,S3client,S3resource = TuringUsersCredentialsControl(id_empresa)
schema_rds = 'EMP_'+id_empresa+'_Proj_'+id_projeto
mysq_conn,engine = TuringRDSManagementConn(id_empresa,schema_rds,turing_adm = 'N')
path_s3_wfile = 's3://'+nm_bucket+'/Projetos/'+'Projeto_'+id_projeto+'/Modelo_Id_'+id_projeto+'/Modeltrain/'

# Caminho do projeto
path_proj = 'Projetos/'+'Projeto_'+id_projeto+'/Modelo_Id_'+id_projeto+'/Modeltrain/'
s3_path = path_s3_wfile
####################################################################################

Acesso RDS Turing Treinamentos


#### Lendo tabela de input para Turing

In [10]:
path_file = 'Projetos/FLAT_TABLES/'+str(nm_file_desenv_orig)
desenv_df = TuringReadS3CSVFile(nm_bucket,path_file,sep=abt_delimiter)
desenv_df.shape

(623, 12)

In [11]:
desenv_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,858,1,"Daly, Mr. Peter Denis",male,51.0,0,0,113055,26.5500,E17,S,1
1,53,1,"Harper, Mrs. Henry Sleeper (Myna Haxtun)",female,49.0,1,0,PC 17572,76.7292,D33,C,1
2,387,3,"Goodwin, Master. Sidney Leonard",male,1.0,5,2,CA 2144,46.9000,NaN,S,0
3,125,1,"White, Mr. Percival Wayland",male,54.0,0,1,35281,77.2875,D26,S,0
4,579,3,"Caram, Mrs. Joseph (Maria Elias)",female,NaN,1,0,2689,14.4583,NaN,C,0


#### Lendo tabela de desenvolvimento do modelo

In [12]:
nm_tab_int = 'abt_desenv.csv'
abt_desenv = TuringReadS3CSVFile(nm_bucket,path_proj+nm_tab_int,sep=abt_delimiter)
abt_desenv.rename( columns={'Unnamed: 0':'id'}, inplace=True )
abt_desenv.shape

(694, 7)

In [13]:
abt_desenv.head()

,id,Fare,Age,Pclass,FamilySize,TGD_Titulo_Mr,target_t_bin
0,793,-0.030371,0.000000,-1.566107,-0.560975,1,0
1,678,0.295895,1.023561,0.827377,3.779968,0,0
2,368,-0.492378,0.000000,0.827377,-0.560975,0,1
3,569,-0.490280,0.177063,0.827377,-0.560975,1,1
4,443,-0.386671,-0.130754,-0.369365,-0.560975,0,1


#### Lendo tabela de Validação

In [14]:
nm_tab_int = 'abt_valida.csv'
abt_valid = TuringReadS3CSVFile(nm_bucket,path_proj+nm_tab_int,sep=abt_delimiter)
abt_valid.rename( columns={'Unnamed: 0':'id'}, inplace=True )
abt_valid.shape

(197, 7)

In [15]:
abt_valid.head()

,id,Fare,Age,Pclass,FamilySize,TGD_Titulo_Mr,target_t_bin
0,665,0.831478,0.177063,-0.369365,0.679295,1,0
1,703,-0.492545,-0.361618,0.827377,-0.560975,1,0
2,306,1.584179,0.000000,-1.566107,-0.560975,0,1
3,127,-0.504626,-0.438572,0.827377,-0.560975,1,1
4,247,-0.356469,-0.438572,-0.369365,0.679295,0,1


#### Parâmetris do Grid Search - busca por melhores hiperparâmetros

In [16]:
nm_tab_int = 'grid_search_param.csv'
grid_search_param = TuringReadS3CSVFile(nm_bucket,path_proj+nm_tab_int,sep=abt_delimiter)
grid_search_param.shape

(10, 71)

#### Lendo tabela de metadados completa - gerada pelo Turing

In [17]:
grid_search_param.head(100)

,Unnamed: 0,C,class_weight,dual,fit_intercept,max_iter,modelo,multi_class,n_jobs,penalty,...,min_child_weight,missing,nthread,objective,reg_alpha,reg_lambda,scale_pos_weight,seed,silent,intercept_scaling
0,0,1.00,None,False,True,100.0,Regressao Logistica,ovr,1.0,l2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0.01,None,NaN,NaN,-1.0,Support Vector Classification,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,NaN,None,NaN,True,100.0,Perceptron,NaN,1.0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,NaN,None,NaN,NaN,NaN,Random Forest Classifier,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,NaN,None,NaN,NaN,NaN,Decision Tree Classifier,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0,NaN,NaN,NaN,NaN,NaN,k-nearest neighbors vote,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0,NaN,None,NaN,True,100.0,Linear Classifiers with SGD,NaN,1.0,l2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0,NaN,NaN,NaN,NaN,NaN,Gradient Boosting,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0,NaN,NaN,NaN,NaN,NaN,XGBoost,NaN,1.0,NaN,...,1.0,None,None,binary:logistic,0.0,1.0,1.0,None,True,NaN
9,0,1.00,None,True,True,1000.0,Linear SVC,ovr,NaN,l2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [18]:
nm_tab_int = 'db_metadados_full.csv'
db_metadados_full = TuringReadS3CSVFile(nm_bucket,path_proj+nm_tab_int,sep=abt_delimiter)
db_metadados_full.shape

(14, 22)

In [19]:
db_metadados_full

,Unnamed: 0,index,Features,Role,Level,Keep,Tipo,Cardinality,qt_missing,pc_miss,...,unique,top,freq,mean,std,min,25%,50%,75%,max
0,0,0,PassengerId,id,ordinal,1,int64,891,0,0,...,NaN,NaN,NaN,446.000000,257.353842,1.00,223.5000,446.0000,668.5,891.0000
1,1,1,Survived,target,binary,1,int64,2,0,0,...,NaN,NaN,NaN,0.383838,0.486592,0.00,0.0000,0.0000,1.0,1.0000
2,2,2,Pclass,input,ordinal,1,int64,3,0,0,...,NaN,NaN,NaN,2.308642,0.836071,1.00,2.0000,3.0000,3.0,3.0000
3,3,3,Sex,input,nominal,1,object,2,0,0,...,2.0,male,577.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,Age,input,interval,1,float64,88,177,19,...,NaN,NaN,NaN,29.699118,14.526497,0.42,20.1250,28.0000,38.0,80.0000
5,5,5,SibSp,input,ordinal,1,int64,7,0,0,...,NaN,NaN,NaN,0.523008,1.102743,0.00,0.0000,0.0000,1.0,8.0000
6,6,6,Parch,input,ordinal,1,int64,7,0,0,...,NaN,NaN,NaN,0.381594,0.806057,0.00,0.0000,0.0000,0.0,6.0000
7,7,7,Ticket,input,nominal,1,object,681,0,0,...,681.0,CA. 2343,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,8,Fare,input,interval,1,float64,248,0,0,...,NaN,NaN,NaN,32.204208,49.693429,0.00,7.9104,14.4542,31.0,512.3292
9,9,9,Cabin,input,nominal,1,object,147,687,77,...,147.0,C23 C25 C27,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Lendo tabela com as variaveis do modelo

In [20]:
nm_tab_int = 'turing_variable_modelo.csv'
turing_variable_modelo = TuringReadS3CSVFile(nm_bucket,path_proj+nm_tab_int,sep=abt_delimiter)
turing_variable_modelo.shape

(5, 3)

In [21]:
turing_variable_modelo

,Unnamed: 0,Variaveis_Modelo,Importancia
0,0,Fare,0.174
1,1,Age,0.083
2,2,Pclass,0.115
3,3,FamilySize,0.068
4,4,TGD_Titulo_Mr,0.433


#### Lendo tabela com lista de variáveis excluídas por quantidade de missing

In [22]:
nm_tab_int = 'lista_vars_missing_rejected.csv'
lista_vars_missing_rejected = TuringReadS3CSVFile(nm_bucket,path_proj+nm_tab_int,sep=abt_delimiter)
lista_vars_missing_rejected.shape

(0, 22)

In [23]:
lista_vars_missing_rejected

,Unnamed: 0,index,Features,Role,Level,Keep,Tipo,Cardinality,qt_missing,pc_miss,...,unique,top,freq,mean,std,min,25%,50%,75%,max


#### Lendo tabela original de entrada

In [24]:
path_file = path_proj+'train_df_full.parquet'
train_df_full = TuringReadParquetFile(S3fs,nm_bucket,path_file)
train_df_full.shape

(891, 14)

In [25]:
train_df_full.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Titulo,FamilySize,IsAlone
0,1,0,3,male,22.0,1,0,A/5 21171,7.2500,None,S,Mr,2,0
1,2,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs,2,0
2,3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,None,S,Miss,1,1
3,4,1,1,female,35.0,1,0,113803,53.1000,C123,S,Mrs,2,0
4,5,0,3,male,35.0,0,0,373450,8.0500,None,S,Mr,1,1


#### Lendo tabela pós data prep

In [26]:
path_file = path_proj+'turing_dataprep.parquet'
turing_dataprep = TuringReadParquetFile(S3fs,nm_bucket,path_file)
turing_dataprep.shape

(891, 19)

In [27]:
turing_dataprep.head()

,Fare,IsAlone,Age,Parch,Pclass,FamilySize,SibSp,TGD_Titulo_Master,TGD_Titulo_Miss,TGD_Titulo_Mr,TGD_Titulo_Mrs,TGD_Titulo_Raros,TGD_Sex_female,TGD_Sex_male,TGD_Embarked_C,TGD_Embarked_Q,TGD_Embarked_S,Survived,WoE_REJECT
0,-0.502445,-1.231645,-0.592481,-0.473674,0.827377,0.059160,0.432793,0,0,1,0,0,0,1,0,0,1,0,0
1,0.786845,-1.231645,0.638789,-0.473674,-1.566107,0.059160,0.432793,0,0,0,1,0,1,0,1,0,0,1,0
2,-0.488854,0.811922,-0.284663,-0.473674,0.827377,-0.560975,-0.474545,0,1,0,0,0,1,0,0,0,1,1,0
3,0.420730,-1.231645,0.407926,-0.473674,-1.566107,0.059160,0.432793,0,0,0,1,0,1,0,0,0,1,1,0
4,-0.486337,0.811922,0.407926,-0.473674,0.827377,-0.560975,-0.474545,0,0,1,0,0,0,1,0,0,1,0,0


#### Lendo tabela com quadro comparativo dos modelos treinados

In [28]:
nm_tab_int = 'model_comparison.csv'
model_comparison = TuringReadS3CSVFile(nm_bucket,path_proj+nm_tab_int,sep=abt_delimiter)
model_comparison.shape

(11, 9)

In [29]:
model_comparison[['Model','Gini','AUCP','AUCR','Tempo_sec']]

,Model,Gini,AUCP,AUCR,Tempo_sec
0,Gradient Boosting Classifier,0.7596,0.8257,0.8798,3.31
1,XGBoost Classifier,0.7254,0.8407,0.8627,0.90
2,Logistic Regression,0.7194,0.8458,0.8597,0.03
3,Linear SVC,0.7184,0.8421,0.8592,0.04
4,Support Vector Machines,0.7038,0.8347,0.8519,0.31
5,Stochastic Gradient Descent,0.6966,0.8114,0.8483,0.01
6,Perceptron,0.6950,0.7821,0.8475,1.32
7,Random Forest,0.6914,0.8327,0.8457,1.18
8,Decision Tree,0.6838,0.7167,0.8419,0.25
9,Naive Bayes,0.6516,0.7416,0.8258,0.01
